In [1]:
import pandas_ip as ip
import numpy as np
import pandas as pd
from pandas.core.internals import BlockManager
import ipaddress

## What currently works:

- Creating IP arrays
- Storing IP arrays in Blocks
- Making Series / DataFrames from blocks

## What kind-of Works:

- with a patch to pandas https://github.com/pandas-dev/pandas/issues/14781

## What doesn't work

- Passing our array to pandas' constructors
- indexing
- everything else :)

## Examples

Our container for ip addresses

In [4]:
values = ip.IPAddress.from_pyints([1, 2])
values

<IPAddress(['0.0.0.1', '0.0.0.2'])>

And convert that to a `Series`.

In [3]:
s = values.to_series()
s

0    0.0.0.1
1    0.0.0.2
dtype: ip

And a DataFrame?

In [4]:
values.to_series().to_frame()

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

whomp whomp. Here's what "should" happen.

In [11]:
blocks = (ip.IPBlock(values, placement=slice(0, 1)),)
bm = BlockManager(blocks, [pd.Index(['A']), pd.RangeIndex(0, 2)])
df = pd.DataFrame(bm)
df

,A
0,0.0.0.1
1,0.0.0.2


The accessor

In [13]:
s.ip.is_ipv4

0    True
1    True
dtype: bool

In [14]:
s.ip.is_ipv6

0    False
1    False
dtype: bool

In [15]:
s.ip.isna

0    False
1    False
dtype: bool

In [16]:
pd.concat([s, s])

0    0.0.0.1
1    0.0.0.2
0    0.0.0.1
1    0.0.0.2
dtype: ip

In [17]:
s.ip.is_ipv4

0    True
1    True
dtype: bool

In [15]:
s.loc[0]

<IPAddress(['0.0.0.1'])>

In [16]:
s.loc[[0]]

AttributeError: 'IPAddress' object has no attribute 'flags'